![Cleaning data](Cleansing-Data.png)

## Six steps to clean data

Clean the **data structure**

1. **Scrape** the document (into a DataFrame)
2. Delete extra **rows**
3. Fix **column** structure

Clean the **data content**

4. Handle **missing** values
5. **Standardize** values
6. **Extract** unstructured data

1. Digitize the document. (format: PDF/Word/..., encoding: UTF-8/CP1252/...)
1. Delete extra rows (header, footer, total, subtotal, column number indicators, blank)
1. Fix column structure (missing headers, rename columns, mis-alignment)
1. Handle missing / duplicate values
  - De-duplicate rows (based on subset of data)
  - Identify (e.g. blank, NA, XX, 999, etc)
  - Ignore missing values
  - Remove rows with NA (subsetting) or columns with NA
  - Fill missing values:
    - Constant (e.g. zero, today, etc)
    - Column (e.g. created date  updated date)
    - Function (e.g. average of rows/columns)
    - External data
1. Standardize values
  - Convert text to numbers or dates
  - Remove extra characters (commas from numbers, prefixes, suffixes, spaces)
  - Correct mis-spellings
1. Extract unstructured data
  - Extract numbers with units (lbs & kgs, kph & mph)
  - Extract lists (e.g. multiple values in cells)
  - Extract dates (e.g. 25/10/16 or Oct 25, 2016)
  - Extract entities (e.g. names, places, phone numbers, PIN codes, things)
  - Extract name structure (e.g. title, first name, last name)

## Let's clean this analyst report

[Open file](Cleansing-Data-ABN-AMRO-report.pdf)

![ABN Amro Analyst Report](Cleansing-Data-ABN-Maro-report.png)

## 1. Scrape the document

Convert data in formats like PDF, HTML, Word, etc into a Python DataFrame

In [289]:
import tabula
data1 = tabula.read_pdf('Cleansing-Data-ABN-AMRO-report.pdf', pages=3)
data1

Unnamed: 0     RWAs    Capital Net profit Net profit.1  \
0                    NaN   (2005)  allocated      2004A        2005A   
1            Netherlands   47,300      2,838        229          509   
2       USA - commercial   66,700      4,002        488          766   
3         USA - mortgage      NaN        NaN        182          105   
4                 Brazil   14,800        888        302          644   
5     New Growth Markets    6,500        390        378          213   
6              Bouwfonds   25,900      1,554        265          315   
7              Wholesale   77,000      4,620        (7)          705   
8        Private Clients    8,100        368        169          255   
9       Asset Management      NaN        451         97          171   
10           Antonveneta   37,000      2,220         na           na   
11  Operating businesses  283,300     17,331      2,103        3,683   
12      Corporate centre    8,600         na      1,584          619   
13        Private equity    3,200      2,711        429          335   
14       Capital surplus       na    (7,178)      (251)        (251)   
15                 Group  295,100     12,864      3,865        4,386   
16          Shares outs.      NaN        NaN        NaN          NaN   
17             Per share      NaN        NaN        NaN          NaN   

   Net profit.2 Net profit.3   As % Peer group Valuation As %.1  
0         2006E        2007E  2006E     PE 06E       NaN    NaN  
1           707          736    16%       11.8     8,345    16%  
2           836          898    19%       14.0    11,700    23%  
3           146          160     3%        9.3     1,364     3%  
4           574          601    13%       11.0     6,336    13%  
5           197          215     4%       11.0     2,171     4%  
6           129          123     3%       11.8     1,523     3%  
7           550          582    12%        8.4     4,620     9%  
8           329          385     7%       17.2     5,637    11%  
9           180          193     4%       17.2     3,095     6%  
10          508          555    11%       12.9     6,580    13%  
11        4,156        4,448    93%       12.4    51,370   101%  
12          305          146    NaN       12.4     3,765    NaN  
13          280          280    NaN         na     2,711    NaN  
14        (251)        (251)    NaN         na   (7,178)    NaN  
15        4,489        4,623    NaN       11.3    50,669    NaN  
16          NaN          NaN    NaN        NaN     1,878    NaN  
17          NaN          NaN    NaN        NaN      27.0    NaN

## 2. Delete extra rows

Header, footer, total, subtotal, column number indicators, blank rows, etc.

In [209]:
data2 = (data1
         .set_index('Unnamed: 0')
         .drop([pd.np.nan, 'Operating businesses', 'Group', 'Shares outs.', 'Per share'])
        )
data2

RWAs  Capital Net profit Net profit.1 Net profit.2  \
Unnamed: 0                                                                 
Netherlands         47,300    2,838        229          509          707   
USA - commercial    66,700    4,002        488          766          836   
USA - mortgage         NaN      NaN        182          105          146   
Brazil              14,800      888        302          644          574   
New Growth Markets   6,500      390        378          213          197   
Bouwfonds           25,900    1,554        265          315          129   
Wholesale           77,000    4,620        (7)          705          550   
Private Clients      8,100      368        169          255          329   
Asset Management       NaN      451         97          171          180   
Antonveneta         37,000    2,220         na           na          508   
Corporate centre     8,600       na      1,584          619          305   
Private equity       3,200    2,711        429          335          280   
Capital surplus         na  (7,178)      (251)        (251)        (251)   

                   Net profit.3 As % Peer group Valuation As %.1  
Unnamed: 0                                                        
Netherlands                 736  16%       11.8     8,345    16%  
USA - commercial            898  19%       14.0    11,700    23%  
USA - mortgage              160   3%        9.3     1,364     3%  
Brazil                      601  13%       11.0     6,336    13%  
New Growth Markets          215   4%       11.0     2,171     4%  
Bouwfonds                   123   3%       11.8     1,523     3%  
Wholesale                   582  12%        8.4     4,620     9%  
Private Clients             385   7%       17.2     5,637    11%  
Asset Management            193   4%       17.2     3,095     6%  
Antonveneta                 555  11%       12.9     6,580    13%  
Corporate centre            146  NaN       12.4     3,765    NaN  
Private equity              280  NaN         na     2,711    NaN  
Capital surplus           (251)  NaN         na   (7,178)    NaN

# 3. Fix column structure

Missing header row, column names split across rows / columns, mis-aligned columns, etc

In [210]:
data3 = data2.copy()
data3.columns = ['RWAs', 'Capital allocated', 'Net profit 2004A', 'Net profit 2005A', 'Net profit 2006E', 'Net profit 2007E', 'As % 2006E', 'Peer group PE06E', 'Valuation', 'As %']
data3.index.name = 'Business'
data3

RWAs Capital allocated Net profit 2004A  \
Business                                                        
Netherlands         47,300             2,838              229   
USA - commercial    66,700             4,002              488   
USA - mortgage         NaN               NaN              182   
Brazil              14,800               888              302   
New Growth Markets   6,500               390              378   
Bouwfonds           25,900             1,554              265   
Wholesale           77,000             4,620              (7)   
Private Clients      8,100               368              169   
Asset Management       NaN               451               97   
Antonveneta         37,000             2,220               na   
Corporate centre     8,600                na            1,584   
Private equity       3,200             2,711              429   
Capital surplus         na           (7,178)            (251)   

                   Net profit 2005A Net profit 2006E Net profit 2007E  \
Business                                                                
Netherlands                     509              707              736   
USA - commercial                766              836              898   
USA - mortgage                  105              146              160   
Brazil                          644              574              601   
New Growth Markets              213              197              215   
Bouwfonds                       315              129              123   
Wholesale                       705              550              582   
Private Clients                 255              329              385   
Asset Management                171              180              193   
Antonveneta                      na              508              555   
Corporate centre                619              305              146   
Private equity                  335              280              280   
Capital surplus               (251)            (251)            (251)   

                   As % 2006E Peer group PE06E Valuation As %  
Business                                                       
Netherlands               16%             11.8     8,345  16%  
USA - commercial          19%             14.0    11,700  23%  
USA - mortgage             3%              9.3     1,364   3%  
Brazil                    13%             11.0     6,336  13%  
New Growth Markets         4%             11.0     2,171   4%  
Bouwfonds                  3%             11.8     1,523   3%  
Wholesale                 12%              8.4     4,620   9%  
Private Clients            7%             17.2     5,637  11%  
Asset Management           4%             17.2     3,095   6%  
Antonveneta               11%             12.9     6,580  13%  
Corporate centre          NaN             12.4     3,765  NaN  
Private equity            NaN               na     2,711  NaN  
Capital surplus           NaN               na   (7,178)  NaN

# 4. Handle missing values

- Identify missing values (blank, NA, XX, 999, ...)
- Plan a strategy to handle them
  - Do nothing
  - Remove rows / columns with missing values
  - Fill with a constant (e.g. zero, today, etc)
  - Fill from another column (e.g. defaults)
  - Fill using a function (e.g. average)
  - Fill with external data

In [211]:
data4 = data3.replace('na', pd.np.nan)
data4

RWAs Capital allocated Net profit 2004A  \
Business                                                        
Netherlands         47,300             2,838              229   
USA - commercial    66,700             4,002              488   
USA - mortgage         NaN               NaN              182   
Brazil              14,800               888              302   
New Growth Markets   6,500               390              378   
Bouwfonds           25,900             1,554              265   
Wholesale           77,000             4,620              (7)   
Private Clients      8,100               368              169   
Asset Management       NaN               451               97   
Antonveneta         37,000             2,220              NaN   
Corporate centre     8,600               NaN            1,584   
Private equity       3,200             2,711              429   
Capital surplus        NaN           (7,178)            (251)   

                   Net profit 2005A Net profit 2006E Net profit 2007E  \
Business                                                                
Netherlands                     509              707              736   
USA - commercial                766              836              898   
USA - mortgage                  105              146              160   
Brazil                          644              574              601   
New Growth Markets              213              197              215   
Bouwfonds                       315              129              123   
Wholesale                       705              550              582   
Private Clients                 255              329              385   
Asset Management                171              180              193   
Antonveneta                     NaN              508              555   
Corporate centre                619              305              146   
Private equity                  335              280              280   
Capital surplus               (251)            (251)            (251)   

                   As % 2006E Peer group PE06E Valuation As %  
Business                                                       
Netherlands               16%             11.8     8,345  16%  
USA - commercial          19%             14.0    11,700  23%  
USA - mortgage             3%              9.3     1,364   3%  
Brazil                    13%             11.0     6,336  13%  
New Growth Markets         4%             11.0     2,171   4%  
Bouwfonds                  3%             11.8     1,523   3%  
Wholesale                 12%              8.4     4,620   9%  
Private Clients            7%             17.2     5,637  11%  
Asset Management           4%             17.2     3,095   6%  
Antonveneta               11%             12.9     6,580  13%  
Corporate centre          NaN             12.4     3,765  NaN  
Private equity            NaN              NaN     2,711  NaN  
Capital surplus           NaN              NaN   (7,178)  NaN

# 5. Standardize values

- Convert text to numbers or dates
- Remove extra characters (prefixes, suffixes, spaces)
- Correct mis-spellings

In [334]:
data5 = (data4
        .replace(',', '', regex=True)                # Remove commas
        .replace('\((\d+)\)', r'-\1', regex=True)    # Convert (10) to -10
        .replace('%$', '', regex=True)               # Convert % to numbers
        .astype(float)                               # Convert to numbers
        )
data5.sum()

RWAs                 295100.0
Capital allocated     12864.0
Net profit 2004A       3865.0
Net profit 2005A       4386.0
Net profit 2006E       4490.0
Net profit 2007E       4623.0
As % 2006E               92.0
Peer group PE06E        137.0
Valuation             50669.0
As %                    101.0
dtype: float64

## 6. Extract unstructured data

- Extract numbers with units (lbs & kgs, kph & mph)
- Extract lists (e.g. multiple values in cells)
- Extract dates (e.g. 25/10/16 or Oct 25, 2016)
- Extract entities (e.g. names, places, phone numbers, PIN codes, things)
- Extract name structure (e.g. title, first name, last name)

This dataset has none of those. Let's pick another dataset.

# Product dosage composition

This is a [random dataset](http://ravenbhelpharma.com/composition.xlsx) I found yesterday.

![Product dosage composition](Cleansing-Data-Pharma.png)

## Let's clean the data

In [259]:
import pandas as pd
pharma = pd.read_excel('Cleansing-Data-composition.xlsx', skiprows=[0, 1]).set_index('S. No.')
pharma.head()

Product              Dosage Form  \
S. No.                                               
1                 Osta-GM                   Tablet   
2       Revidec + Osta-D3  Combipack of injections   
3              Osta-D Cal                   Tablet   
4              Osta-D Cal               Suspension   
5            Exudase-TH 8                   Tablet   

                                              Composition  
S. No.                                                     
1       Diacerein 50 mg, Glucosamine sulphate 750 mg, ...  
2       Nandrolone decanoate 50 mg injection + Choleca...  
3       Alfacalcidol 0.25 mcg, Calcium carbonate 625 m...  
4       Calcium (as CaCO3) 150 mg, Magnesium 25 mg, Zi...  
5       Thiocolchicoside 8 mg, Diclofenac potassium 50 mg

1. **Scrape** the document: **Done**. `pd.read_excel`.
2. Delete extra **rows**: **Done**. `skiprows=[0, 1]`
3. Fix **column** structure: **None**. It's OK

as well as in data **content**

4. Handle **missing** values: **None**.
5. **Standardize** values: TODO
6. **Extract** unstructured data: TODO

## 5. Standardize values

The "Dosage Form" column is not standardized.

In [260]:
pharma.sort_values('Dosage Form', ascending=True)

Product                            Dosage Form  \
S. No.                                                                
37                 Zopra-DSR                                Capsule   
23                Odpant-DSR                                Capsule   
13                  Zintra-G                                Capsule   
33                   Indopra                               Capsule    
47                  Ezycart                                Capsule    
24                       SFe                        Chewable tablet   
2          Revidec + Osta-D3                Combipack of injections   
39                   Revicof                            Cough Syrup   
19             Revibrox Plus                            Cough Syrup   
18                  Revispas                                  Drops   
45              Hactamoz 457                              Dry syrup   
48      Testorich injection           Each 1 ml injection contains:   
49            Testorich-100           Each 1 ml injection contains:   
55         Revibrox-LS syrup                    Each 5 ml contains:   
53                 Ornapreg   Each eneteric coated tablet contains:   
44              Hactamox-625     Each film coated Tablets contains:   
41                 Osta heal    Each hard gelatin capsule contains:   
56         Revibrox-LS drops                       Each ml contain:   
40                   Emergel                      Each ml contains:   
30                    Dolfre                                  Gel     
27                 Modest-45                              Injection   
21                    Hb-Red                              Injection   
17                   Osta-D3                            Massage Oil   
31                    Dolfre                                    Oil   
54                    Acolax                          Oral solution   
26                  Pubicare                          Perineal Wash   
14             Zintra Powder                                 Powder   
38                       XND    Roll on for use in nasal congestion   
15                  Osta-Eve                    Softgelatin Capsule   
16                    Osta-P                             Suspension   
4                 Osta-D Cal                             Suspension   
12                  Zintra-M                                  Syrup   
43              Hactaflox CF                                 Tablet   
42               Hactflox OZ                                 Tablet   
36                 Rucadol-P                                 Tablet   
1                    Osta-GM                                 Tablet   
34                Tizalud-DP                                 Tablet   
3                 Osta-D Cal                                 Tablet   
5               Exudase-TH 8                                 Tablet   
6                 Exudase-TH                                 Tablet   
7                 Exudase-DP                                 Tablet   
8                 Exudase-D                                  Tablet   
35                Rucadol-RB                                 Tablet   
9                    Gabacom                                 Tablet   
28               Revitrol-K2                                 Tablet   
11                Gabacom-NT                                 Tablet   
20                    Hb-Red                                 Tablet   
22                 Odpant-DM                                 Tablet   
32               Dolfre Plus                                 Tablet   
10             Gabacom M 100                                 Tablet   
46               Hactamox-DS                                Tablets   
25                    Cans-3                    Vaginal Suppository   
50                    Camful                                capsule   
51                   Zopra-L                                capsule   
52            Revibact forte                 

## 5. Standardize values with fingerprints

In [338]:
from jellyfish import metaphone, soundex, nysiis, match_rating_codex
metaphone('tablot')

'TBLT'

In [339]:
pharma['Dosage Form'].apply(metaphone)

S. No.
1                          TBLT
2              KMBPK OF INJKXNS
3                          TBLT
4                        SSPNXN
5                          TBLT
6                          TBLT
7                          TBLT
8                          TBLT
9                          TBLT
10                         TBLT
11                         TBLT
12                          SRP
13                         KPSL
14                          PTR
15                 SFTJLTN KPSL
16                       SSPNXN
17                       MSJ OL
18                         TRPS
19                        K SRP
20                         TBLT
21                       INJKXN
22                         TBLT
23                         KPSL
24                    XWBL TBLT
25                   FJNL SPSTR
26                      PRNL WX
27                       INJKXN
28                         TBLT
29                       SSPNXN
30                          JL 
31                           OL
3

In [341]:
pharma.groupby(pharma['Dosage Form'].apply(soundex))['Dosage Form'].unique().tolist()

[array(['Capsule', 'Capsule ', 'capsule'], dtype=object),
 array(['Chewable tablet'], dtype=object),
 array(['Cough Syrup'], dtype=object),
 array(['Combipack of injections'], dtype=object),
 array(['Drops'], dtype=object),
 array(['Dry syrup'], dtype=object),
 array(['Each film coated Tablets contains:'], dtype=object),
 array(['Each eneteric coated tablet contains:'], dtype=object),
 array(['Each ml contains:', 'Each 1 ml injection contains:',
        'Each 5 ml contains:', 'Each ml contain:'], dtype=object),
 array(['Each hard gelatin capsule contains:'], dtype=object),
 array(['Gel  '], dtype=object),
 array(['Injection', 'injection'], dtype=object),
 array(['Massage Oil'], dtype=object),
 array(['Oil'], dtype=object),
 array(['Oral solution'], dtype=object),
 array(['Powder'], dtype=object),
 array(['Perineal Wash'], dtype=object),
 array(['Roll on for use in nasal congestion'], dtype=object),
 array(['Softgelatin Capsule'], dtype=object),
 array(['Suspension', 'suspension'], dtyp

## 5. Standardize values: custom fingerprints

You can create your own fuzzy matching algorithms too.

For example, let's listen to a few [Hindi songs from AR Rahman](http://www.s-anand.net/blog/a-r-rahman-hindi-songs/)

![Hindi Songs Quiz](Cleansing-Data-Hindi-Songs.png)

## 5. Standardize values: custom fingerprints

For this, I used a custom fingerprinting algorithm suited for Hindi movies and artistes.

In [265]:
def fingerprint_hindi(s):    
        s = s + ' '
        s = s.upper()                               # Upper case
        s = re.sub(r'\s+', ' ', s)                  # Single space
        s = re.sub(r'W', 'V', s)                    # Vivaah or Wivaah?
        s = re.sub(r'KSH', 'X', s)                  # Dixit or Deekshit?
        s = re.sub(r'Z', 'J', s)                    # Jabardast or Zabardast?
        s = re.sub(r'PH', 'F', s)                   # Phirangi or Firangi?
        s = re.sub(r'([KGCJTDPBS])H', r'\1', s)     # Dheeraj or Diraj?
        s = re.sub(r'(.)\1', r'\1', s)              # Kaala or Kala?
        s = re.sub(r'Y', '', s)                     # Koyi or Koi?
        s = re.sub(r'([AEIOU])N ', r'\1', s)        # Hein or Hai?
        s = re.sub(r'[AEIOU]', '', s)               # Rehman or Rahman?
        s = re.sub(r' ', '', s)                     # Remove spaces
        return s

In [304]:
assert fingerprint_hindi('Meenakshi Dixit') == fingerprint_hindi('minaxi  deekshith') == 'MNXDXT'
assert fingerprint_hindi('Ye Dil Hai Mushkil') == fingerprint_hindi('e dil hein muskil') == 'DLHMSKL'

In [1]:
# Here's an example of the OpenRefine fingerprint algorithm 
# Source: https://gist.github.com/cjdd3b/0386f139bb953f046c6e
import re
import string
from unidecode import unidecode    # install this

PUNCTUATION = re.compile('[%s]' % re.escape(string.punctuation))

def preprocess(string):
    return PUNCTUATION.sub('', string.strip().lower())

def unique_preserving_order(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
        
def fingerprint(s):
    s = preprocess(s)
    return unidecode(' '.join(unique_preserving_order(sorted(s.split()))))

def ngram_fingerprint(s, n=1):
    s = preprocess(s)
    return unidecode(''.join(unique_preserving_order(sorted([s[i:i + n] for i in range(len(s) - n + 1)]))))

## 5. Standardize values: constituencies

Let's apply fingerprinting to the list of assembly constituency names across years.

In [285]:
assembly = pd.read_csv('D:/site/gramener.com/viz/india-election-data/assembly-elections/assembly.csv')
assembly

C:\anaconda\3.6\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ST_NAME  YEAR  AC_NO         AC_NAME AC_TYPE  #  \
0       Andhra Pradesh  1955      1       ICHAPURAM     GEN  1   
1       Andhra Pradesh  1955      1       ICHAPURAM     GEN  2   
2       Andhra Pradesh  1955      1       ICHAPURAM     GEN  3   
3       Andhra Pradesh  1955      1       ICHAPURAM     GEN  4   
4       Andhra Pradesh  1955      1       ICHAPURAM     GEN  5   
5       Andhra Pradesh  1955      2         SOMPETA     GEN  1   
6       Andhra Pradesh  1955      2         SOMPETA     GEN  2   
7       Andhra Pradesh  1955      2         SOMPETA     GEN  3   
8       Andhra Pradesh  1955      3  BRAHMANATARALA     GEN  1   
9       Andhra Pradesh  1955      3  BRAHMANATARALA     GEN  2   
10      Andhra Pradesh  1955      3  BRAHMANATARALA     GEN  3   
11      Andhra Pradesh  1955      3  BRAHMANATARALA     GEN  4   
12      Andhra Pradesh  1955      3  BRAHMANATARALA     GEN  5   
13      Andhra Pradesh  1955      3  BRAHMANATARALA     GEN  6   
14      Andhra Pradesh  1955      4         TEKKALI     GEN  1   
15      Andhra Pradesh  1955      4         TEKKALI     GEN  2   
16      Andhra Pradesh  1955      4         TEKKALI     GEN  3   
17      Andhra Pradesh  1955      4         TEKKALI     GEN  4   
18      Andhra Pradesh  1955      4         TEKKALI     GEN  5   
19      Andhra Pradesh  1955      4         TEKKALI     GEN  6   
20      Andhra Pradesh  1955      5   NARASANNAPETA     GEN  1   
21      Andhra Pradesh  1955      5   NARASANNAPETA     GEN  2   
22      Andhra Pradesh  1955      5   NARASANNAPETA     GEN  3   
23      Andhra Pradesh  1955      5   NARASANNAPETA     GEN  4   
24      Andhra Pradesh  1955      6      PATAPATNAM     GEN  1   
25      Andhra Pradesh  1955      6      PATAPATNAM     GEN  2   
26      Andhra Pradesh  1955      6      PATAPATNAM     GEN  3   
27      Andhra Pradesh  1955      6      PATAPATNAM     GEN  4   
28      Andhra Pradesh  1955      6      PATAPATNAM     GEN  5   
29      Andhra Pradesh  1955      6      PATAPATNAM     GEN  6   
...                ...   ...    ...             ...     ... ..   
391461     West Bengal  2011    290      MAYURESWAR     GEN  1   
391462     West Bengal  2011    290      MAYURESWAR     GEN  2   
391463     West Bengal  2011    290      MAYURESWAR     GEN  3   
391464     West Bengal  2011    291       RAMPURHAT     GEN  1   
391465     West Bengal  2011    291       RAMPURHAT     GEN  2   
391466     West Bengal  2011    291       RAMPURHAT     GEN  3   
391467     West Bengal  2011    291       RAMPURHAT     GEN  4   
391468     West Bengal  2011    291       RAMPURHAT     GEN  5   
391469     West Bengal  2011    292          HANSAN     GEN  1   
391470     West Bengal  2011    292          HANSAN     GEN  2   
391471     West Bengal  2011    292          HANSAN     GEN  3   
391472     West Bengal  2011    292          HANSAN     GEN  4   
391473     West Bengal  2011    292          HANSAN     GEN  5   
391474     West Bengal  2011    292          HANSAN     GEN  6   
391475     West Bengal  2011    292          HANSAN     GEN  7   
391476     West Bengal  2011    292          HANSAN     GEN  8   
391477     West Bengal  2011    293         NALHATI     GEN  1   
391478     West Bengal  2011    293         NALHATI     GEN  2   
391479     West Bengal  2011    293         NALHATI     GEN  3   
391480     West Bengal  2011    293         NALHATI     GEN  4   
391481     West Bengal  2011    293         NALHATI     GEN  5   
391482     West Bengal  2011    293         NALHATI     GEN  6   
391483     West Bengal  2011    293         NALHATI     GEN  7   
391484     West Bengal  2011    293         NALHATI     GEN  8   
391485     West Bengal  2011    294         MURARAI     GEN  1   
391486     West Bengal  2011    294         MURARAI     GEN  2   
391487     West Bengal  2011    294         MURARAI     GEN  3   
391488     West Bengal  2011    294         MURARAI     GEN  4   
391489     West Bengal  2011    294         MURARAI    

## 5. Standardize values: constituencies

This corrects phonetic spellings mistakes like:

- BHADRACHALAM, BHADRACHELAM, BHADRAHCALAM
- TUGHLAKABAD, TUGHLAQABAD, TUGHLAQUABAD, TUGLAKABAD
- RISHIVANDIYAM, RISHIVADIYAM, RISHIVANDIAM, RISHIVANDIAM, RISHIVANDIYAM

... and non-phonetic ones like:

- OOTACAMUND, UDHAGAMANDALAM
- POTTAPIDARAM, OTTAPIDARAM
- HARASAPUR, NARSAPUR

... but we were still left with a huge gap.

## A brief detour: Delimitation

Constituencies are defined in rough proportion to population.

- 1952: Constituencies defined using 1951 census
- 1962: Constituencies defined using 1961 census
- 1972: Constituencies defined using 1971 census
- 1976: Stop delimitation until 2000. States promoting family planning don't lose
- 2002: Extend till 2026. But apply intra-state delimitation. Preserve # seats per state
- 2008: Intra-state delimination was actually implemented

For each of these years, we have potential re-definition of constituencies.

The biggest nightmare this caused me was **Machilipatnam**.

## A brief detour: Machilipatnam

![Machilipatnam market](https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Machilipatnam_koneru_center_3.JPG/250px-Machilipatnam_koneru_center_3.JPG)

Has 2 different names:

- Masulipatnam (or Machilipatnam): derived from the Greek trade of "muslin" (plain woven) cloth
- Bandar: derived from the Turkish word for "port"

In 2008, the Bandar constituency was *renamed* to Machilipatnam.

- Machilipatnam was NOT a new constituency
- Bandar was not removed from the list
- So history needs to be preserved

## 5. Standardize values: analysis

This is where the data itself came in handy.

![Assembly elections constituencies](Cleansing-Data-assembly-elections.png)

## 6. Extract values

- Extract numbers with units (lbs & kgs, kph & mph)
- Extract lists (e.g. multiple values in cells)
- Extract dates (e.g. 25/10/16 or Oct 25, 2016)
- Extract entities (e.g. names, places, phone numbers, PIN codes, things)
- Extract name structure (e.g. title, first name, last name)

## 6. Extract values: composition list

In [305]:
pharma['Composition'].head()

S. No.
1    Diacerein 50 mg, Glucosamine sulphate 750 mg, ...
2    Nandrolone decanoate 50 mg injection + Choleca...
3    Alfacalcidol 0.25 mcg, Calcium carbonate 625 m...
4    Calcium (as CaCO3) 150 mg, Magnesium 25 mg, Zi...
5    Thiocolchicoside 8 mg, Diclofenac potassium 50 mg
Name: Composition, dtype: object

In [311]:
components = pharma['Composition'].str.split(r'\s*[,\+]\s*').apply(pd.Series).stack()
components.head(10)

S. No.   
1       0                                      Diacerein 50 mg
        1                          Glucosamine sulphate 750 mg
        2                                           MSM 250 mg
2       0                 Nandrolone decanoate 50 mg injection
        1                   Cholecalciferol 6 lac IU injection
3       0                                Alfacalcidol 0.25 mcg
        1    Calcium carbonate 625 mg eq. to Elemental calc...
4       0                            Calcium (as CaCO3) 150 mg
        1                                      Magnesium 25 mg
        2                                          Zinc 1.5 mg
dtype: object

## 6. Extract values: lists

In [327]:
extracts = components.str.extract('(.*?)\s*(\d+)\s*(ml|mg|mcg|g|gm).*')
extracts['text'] = components
extracts.head()

0    1    2                                  text
S. No.                                                                        
1      0             Diacerein   50   mg                       Diacerein 50 mg
       1  Glucosamine sulphate  750   mg           Glucosamine sulphate 750 mg
       2                   MSM  250   mg                            MSM 250 mg
2      0  Nandrolone decanoate   50   mg  Nandrolone decanoate 50 mg injection
       1                   NaN  NaN  NaN    Cholecalciferol 6 lac IU injection

## Don't overdo it

![Mess](https://imgs.xkcd.com/comics/mess.png)

# Let's automate as open source

We're creating an open source tool on **[Github: gramener/dataaudit](https://github.com/gramener/dataaudit)**.

(See the **dev** branch for now.)

We'll start with cleaning public health data at UP & Bihar.

Join in!